~~~
Copyright 2026 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Analyze Computed Tomography (CT) Images with MedGemma 1.5

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-health/medgemma/blob/main/notebooks/high_dimensional_ct_model_garden.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogle-Health%2Fmedgemma%2Fmain%2Fnotebooks%2Fhigh_dimensional_ct_model_garden.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-health/medgemma/blob/main/notebooks/high_dimensional_ct_model_garden.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/medgemma-release-680aade845f90bec6a3f60c4">
      <img alt="Hugging Face logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on Hugging Face
    </a>
  </td>
</tr></tbody></table>

This notebook demonstrates how 3D representation of [computed tomography (CT)](https://www.nibib.nih.gov/science-education/science-topics/computed-tomography-ct) imaging can be used to prompt MedGemma 1.5 running on Vertex AI. This notebook is for educational purposes only to show the baseline functionality of MedGemma 1.5. It does not represent a finished or approved product, is not intended to diagnose or suggest treatment for any disease or condition, and should not be used for medical advice. See [HAI-DEF Terms of Use](https://developers.google.com/health-ai-developer-foundations/terms) for more information.

Vertex AI makes it easy to serve your model and make it accessible to the world. Learn more about [Vertex AI](https://cloud.google.com/vertex-ai/docs/start/introduction-unified-platform).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing); use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

In [ ]:
# @title Install pydicom Python library

%%capture
! pip install pydicom

In [ ]:
# @title Authenticate with Google Cloud (only needed for Google Colab)
import os
import sys

google_colab = "google.colab" in sys.modules and not os.environ.get("VERTEX_PRODUCT") == "COLAB_ENTERPRISE"

if google_colab:
    from google.colab import auth
    # There will be a popup asking you to sign in with your user account and approve
    # access
    auth.authenticate_user()

## Retrieve Imaging Data Commons (IDC) CT image metadata

[Imaging Data Commons (IDC)](https://datacommons.cancer.gov/repository/imaging-data-commons#) is one of the largest publicly available, de-identified, repositories for cancer imaging. The repository is funded by the [National Cancer Institute (NCI)](https://www.cancer.gov/), an institute of the [National Institutes of Health (NIH)](https://www.nih.gov/), a part of the [U.S. Department of Health and Human Service](https://www.hhs.gov/). IDC contains images for all major medical imaging modalities. Images are stored within the archive as [DICOM](https://www.dicomstandard.org/). Images and associated metadata can be searched and visualized through the [IDC website](https://portal.imaging.datacommons.cancer.gov/explore/) or [BigQuery](https://cloud.google.com/healthcare-api/docs/resources/public-datasets/idc), and can be accessed using DICOMweb ([IDC tutorial](https://learn.canceridc.dev/data/downloading-data/dicomweb-access)). For the purposes of this notebook we have mirrored the required images in a Google DICOM store.

DICOM is the medical imaging format generated by CT scanners.
[DICOM](https://www.dicomstandard.org/) images are uniquely identified by three UIDs: Study Instance UID, Series Instance UID, and a SOP Instance UID.  This notebook retrieves a CT image from IDC by downloading all of the images associated with a CT scan. Conceptually, a Study Instance UID can be thought of as the UID that identifies all images acquired or generated as a result of a patient exam. Each medical image acquired as part of the exam (e.g., CT acquisition) is identified by a unique Series Instance UID. Each image acquired or generated as part of the acquisition is, in turn, identified with a unique SOP Instance UID.

CT imaging is commonly represented as an ordered collection of 2D images (slices). Each slice is represented by a 2D image that describes an axial cross section (volume) of the imaged body.



In [ ]:
# @title Read metadata for the DICOM instances in a series
from typing import Sequence

import google.auth
import google.auth.transport
import pydicom
import requests


def get_auth_token() -> str:
  auth_credentials = google.auth.default(['https://www.googleapis.com/auth/cloud-healthcare'])[0]
  auth_credentials.refresh(google.auth.transport.requests.Request())
  return auth_credentials.token


def get_sop_instance_dicomweb_urls(study_instance_uid: str, series_instance_uid: str) -> Sequence[str]:
  # Read DICOM instance metadata for imaging
  # Get oauth credentials to read from store
  headers = {"Authorization" : f"Bearer {get_auth_token()}"}
  dicomweb_series_url = f"https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/{study_instance_uid}/series/{series_instance_uid}"
  metadata = requests.get(f"{dicomweb_series_url}/instances", headers=headers).json()
  dicom_instances = [pydicom.Dataset.from_json(i) for i in metadata]
  # Order instance metadata by instance number
  dicom_instances = sorted(dicom_instances, key=lambda i: int(i.InstanceNumber))

  # Sample 85 slices uniformly across CT volume
  MAX_SLICE = 85
  if len(dicom_instances) > MAX_SLICE:
    dicom_instances = [dicom_instances[int(round(i /(MAX_SLICE) * (len(dicom_instances)-1)))] for i in range(1, MAX_SLICE + 1)]
  return [f'{dicomweb_series_url}/instances/{dcm.SOPInstanceUID}' for dcm in dicom_instances]

In [ ]:
# @title ## Construct MedGemma 1.5 prompt formatted as Chat Completion

# This notebook uses images hosted in the Imaging Data Commons (IDC) archive
# This notebook utilizes data from The Cancer Imaging Archive (TCIA).
# https://www.cancerimagingarchive.net/collection/colorectal-liver-metastases/
# Liver CRLM - CT 1026
study_instance_uid = "1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436"
series_instance_uid = "1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220"

dicom_source = get_sop_instance_dicomweb_urls(study_instance_uid, series_instance_uid)

instruction = ("You are an instructor teaching medical students. You are "
               "analyzing a contiguous block of CT slices from the center of "
               "the abdomen. Please review the slices provided below "
               "carefully.")
content = [{"type": "text", "text": instruction}]
content.append({"type": "image_dicom", "image_dicom": {"dicom_source": dicom_source, 'access_credential': get_auth_token()}})
query_text = ("\n\nBased on the visual evidence in the slices provided above, "
              "is this image a good teaching example of liver pathology? "
              "Comment on hypodense lesions or other hepatic irregularities. "
              "Do not comment on findings outside the liver. Please provide "
              "your reasoning and conclude with a 'Final Answer: yes' or "
              "'Final Answer: no'.")
content.append({"type": "text", "text": query_text})

instance = {
    "@requestFormat": "chatCompletions",
    "messages": [{"role": "user", "content": content}],
    "max_tokens": 500,
    "temperature": 0
}

In [ ]:
# @title ## Display MedGemma 1.5 prompt
import json
from IPython.display import display, Markdown

def mask_bearer_token(obj):
  """Masks bearer token."""
  if isinstance(obj, dict):
    return {k: mask_bearer_token(v) if k != 'access_credential' else 'TOKEN' for k, v in obj.items()}
  elif isinstance(obj, list):
    return [mask_bearer_token(elem) for elem in obj]
  return obj

txt = json.dumps(mask_bearer_token(instance), indent=4, sort_keys=True)
display(Markdown(f'```json\n{txt}'))

```json
{
    "@requestFormat": "chatCompletions",
    "max_tokens": 500,
    "messages": [
        {
            "content": [
                {
                    "text": "You are an instructor teaching medical students. You are analyzing a contiguous block of CT slices from the center of the abdomen. Please review the slices provided below carefully.",
                    "type": "text"
                },
                {
                    "image_dicom": {
                        "access_credential": "TOKEN",
                        "dicom_source": [
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.475273662596035314302684328529",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.231142193158026555203864746037",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.302297530008610262481602047087",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.190960328130437367326276502376",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.214089864075573237476119770626",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.174332313835818207466803310411",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.180537612945505818606587783969",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.405102839118124449252609177118",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.266761757042739995245700278237",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.227436103971867794261713320202",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.600064396808909835377922556388",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.186939469153882962728793601203",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.170998679375101828913637577724",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.244694012748215200319554940658",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.852414061205359600465498795985",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.209262910331802821465596992536",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.327185406336369507258611871750",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.179687663422977549475519907111",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.181474902463086566230854381436",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.233514649674291726915754979900",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.361532762406708143738372646850",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.830613239463755152804216145996",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.301552068461004453076029701451",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.222517415400664504186695471282",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.313372316268460883375154695008",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.189189473919011968547669685121",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.225044713289996559967925925833",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.755399102190683505013136070851",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.546156832809619538552768705199",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.315161309265302356758091759152",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.110588217517785610957724565828",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.303925255271219615595537739043",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.332089664826268215512300446108",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.119909471920231057343036958985",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.216665802898138558389872772396",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.872011087736177264730437480446",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.837829465261434842508182774331",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.323865487309226783908225736915",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.302572411786247455876129428847",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.219934395340248254875845941851",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.222734114401353419027416481676",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.275630861214679094027634786561",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.266347882205983951341441440857",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.131581330498440215128617078937",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.265806801324990089044925893194",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.249591715516516680330434213730",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.155964817805406786372373755702",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.162655284282573812757193808972",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.306253407785342121139904867166",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.149199060934943003148353214800",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.190402308117504884628191661084",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.145397390350809400014616174867",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.320119403924772097576500249015",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.161655152316742359799872016786",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.820448157649521320746231934821",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.283565907139287625834050240706",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.157231446533457413659128601762",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.107101751675260887276054795590",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.198186304108503975604234239454",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.109081856441304431617859035749",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.215616020765651148269769057924",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.149081807955788195839266371066",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.109724447573146232220262090401",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.129668256345963751797706445020",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.506095083286473705236474997506",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.172291884077510187156668669018",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.117052723010670105722132967581",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.108399401097333823351482255825",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.414296426287777112097850828389",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.228562660281490329234551679302",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.161194399437240717202306532389",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.168771884587837583168007593325",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.686825290080668417559582848168",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.477441187799385355060835584898",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.884193210431343619286615792179",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.307213070982087854088686341690",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.138300959540467713707195195327",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.702927050896958784914824175980",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.376835262753192890804744483788",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.325494907912019801920046789482",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.588215090796023621376225975050",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.327342093919148238560554060312",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.211426399929246327923268702156",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.142106182050391254108946189706",
                            "https://healthcare.googleapis.com/v1/projects/hai-cd3-foundations/locations/us-central1/datasets/public/dicomStores/test-images/dicomWeb/studies/1.3.6.1.4.1.14519.5.2.1.9203.8273.982856921320609617394372605436/series/1.3.6.1.4.1.14519.5.2.1.9203.8273.275179554444442893192427753220/instances/1.3.6.1.4.1.14519.5.2.1.9203.8273.770073332760280770187004140942"
                        ]
                    },
                    "type": "image_dicom"
                },
                {
                    "text": "\n\nBased on the visual evidence in the slices provided above, is this image a good teaching example of liver pathology? Comment on hypodense lesions or other hepatic irregularities. Do not comment on findings outside the liver. Please provide your reasoning and conclude with a 'Final Answer: yes' or 'Final Answer: no'.",
                    "type": "text"
                }
            ],
            "role": "user"
        }
    ],
    "temperature": 0
}

In [ ]:
# @title ## Set up Google Cloud environment and MedGemma 1.5 Vertex AI endpoint

# @markdown #### Prerequisites
# @markdown
# @markdown 1. Make sure that [billing is enabled](https://cloud.google.com/billing/docs/how-to/modify-project) for your project.
# @markdown
# @markdown 2. Make sure that either the Compute Engine API is enabled or that you have the [Service Usage Admin](https://cloud.google.com/iam/docs/understanding-roles#serviceusage.serviceUsageAdmin) (`roles/serviceusage.serviceUsageAdmin`) role to enable the API.
# @markdown
# @markdown This section sets the default Google Cloud project and enables the Compute Engine API (if not already enabled), and initializes the Vertex AI API.

import os
from google.cloud import aiplatform

Google_Cloud_Project = ""  # @param {type: "string", placeholder:"e.g. MyProject"}

# @markdown To get [online inferences](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions), you will need a MedGemma 1.5 [Vertex AI endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment) that has been deployed from Model Garden. If you have not already done so, go to the [MedGemma model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/medgemma) and click **"Deploy model"** to deploy the model.
# @markdown
# @markdown **Note:**
# @markdown - This notebook is intended to be used with MedGemma 1.5. Make sure to deploy and use a MedGemma 1.5 endpoint that is configured with an image limit of at least 85 images (in the Model Garden **"Deploy options"** panel, check the image limit under **"Deployment Settings" > "Serving spec"**).
# @markdown - This notebook requires a [dedicated Vertex AI endpoint](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type). Dedicated endpoints are the default endpoint type for Model Garden deployments.

# @markdown This section gets the Vertex AI endpoint resource that you deployed from Model Garden to use for online inferences.
# @markdown
# @markdown Fill in the endpoint ID and region below. You can find your deployed endpoint on the [Vertex AI endpoints page](https://console.cloud.google.com/vertex-ai/online-prediction/endpoints).

ENDPOINT_ID = ""  # @param {type: "string", placeholder:"e.g. 123456789"}
ENDPOINT_REGION = ""  # @param {type: "string", placeholder:"e.g. us-central1"}

os.environ["CLOUDSDK_CORE_PROJECT"] = Google_Cloud_Project
os.environ["GOOGLE_CLOUD_PROJECT"] = Google_Cloud_Project
os.environ["GOOGLE_CLOUD_REGION"] = ENDPOINT_REGION

# Enable the Compute Engine API, if not already.
print("Enabling Compute Engine API.")
! gcloud services enable compute.googleapis.com

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=os.environ["GOOGLE_CLOUD_PROJECT"],
                location=os.environ["GOOGLE_CLOUD_REGION"],
                api_transport="rest")

endpoint = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_ID,
    project=Google_Cloud_Project,
    location=ENDPOINT_REGION,
)

# Use the endpoint name to check that you are using an appropriate model variant.
# These checks are based on the default endpoint name from the Model Garden
# deployment settings.
ENDPOINT_NAME = endpoint.display_name
# Check that "1.5" (formatted as "1_5") is in the endpoint name
if "1_5" not in ENDPOINT_NAME:
    raise ValueError(
        "The examples in this notebook are intended to be used with MedGemma "
        "1.5. Please deploy and use an endpoint with the MedGemma 1.5 model."
    )

Enabling Compute Engine API.


To take a quick anonymous survey, run:
  $ gcloud survey

Initializing Vertex AI API.


In [ ]:
# @title ## Call MedGemma 1.5 and return prediction

import json
from IPython.display import display, Markdown

response = endpoint.raw_predict(
    body=json.dumps(instance).encode('utf-8'), use_dedicated_endpoint=True,
    headers={'Content-Type': 'application/json'}
)
response.raise_for_status()
medgemma_response = response.json()
try:
  medgemma_response = medgemma_response["choices"][0]["message"]["content"]
  display(Markdown(f"---\n\n**[ MedGemma ]**\n\n{medgemma_response}\n\n---"))
except KeyError:
  print(medgemma_response["error"])


---

**[ MedGemma ]**

Based on the provided CT image, there is a hypodense lesion in the right hepatic lobe. This lesion appears somewhat ill-defined and has a slightly heterogeneous appearance.

The liver parenchyma itself appears generally normal in terms of overall density and contour. There are no obvious signs of cirrhosis, fatty infiltration, or other gross parenchymal abnormalities.

Therefore, this image could serve as a teaching example demonstrating a focal liver lesion.

Final Answer: yes

---

## Next steps

Explore the other [notebooks](https://github.com/google-health/medgemma/blob/main/notebooks) to learn what else you can do with the model.